In [1]:
from tensorflow.keras.datasets import boston_housing

In [2]:
(x_train, y_train), (x_test, y_test) = boston_housing.load_data()

In [3]:
x_train.shape, y_train.shape, x_test.shape, y_test.shape

((404, 13), (404,), (102, 13), (102,))

In [4]:
import pandas as pd

In [5]:
pd.Series(y_train).describe()

count    404.000000
mean      22.395050
std        9.210442
min        5.000000
25%       16.675000
50%       20.750000
75%       24.800000
max       50.000000
dtype: float64

In [6]:
from tensorflow.keras import Model, Input, layers as L

In [7]:
# 定义多元性线回归模型
# 就是没有激活函数的密集层

class Net:
    def __init__(self):
        self.dense = L.Dense(1)
    
    def build_model(self):
        x_in = Input(shape=13)
        out = self.dense(x_in)
        
        return Model(x_in, out)
    
model = Net().build_model()

In [8]:
model.summary()

Model: "model"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         [(None, 13)]              0         
_________________________________________________________________
dense (Dense)                (None, 1)                 14        
Total params: 14
Trainable params: 14
Non-trainable params: 0
_________________________________________________________________


In [9]:
model.compile(optimizer='adam', loss='mse', metrics=['mse'])

In [26]:
# 内容较多，注释了输出信息
%time model.fit(x_train, y_train, epochs=10000, batch_size=32)

In [11]:
x_test[10,:].reshape(1, -1)

array([[2.2876e-01, 0.0000e+00, 8.5600e+00, 0.0000e+00, 5.2000e-01,
        6.4050e+00, 8.5400e+01, 2.7147e+00, 5.0000e+00, 3.8400e+02,
        2.0900e+01, 7.0800e+01, 1.0630e+01]])

In [12]:
y_test[54:57], model.predict(x_test[54:57])

(array([34.9, 26.6,  7.2]),
 array([[33.339603],
        [26.987555],
        [19.277012]], dtype=float32))

## 使用pytorch搭建多元线性回归

In [13]:
import torch as t
from torch import nn  
import torch.nn.functional as F

In [14]:
from tensorflow.keras.datasets import boston_housing
(x_train, y_train), (x_test, y_test) = boston_housing.load_data()

In [15]:
class MultiRegression(nn.Module):
    def __init__(self):
        super(MultiRegression, self).__init__()
        self.linear = nn.Linear(13, 1)
    
    def forward(self, x):
        y = self.linear(x)
        return y
    
regression = MultiRegression()

In [16]:
regression

MultiRegression(
  (linear): Linear(in_features=13, out_features=1, bias=True)
)

In [17]:
opt = t.optim.Adam(regression.parameters()) # 注意学习率的大小

In [18]:
criterion = nn.MSELoss()

In [19]:
x_t = t.from_numpy(x_train).float()
y_t = t.from_numpy(y_train).float()

In [20]:
y_t = y_t.reshape(-1, 1)

In [21]:
y_pred = regression(x_t)
loss = criterion(y_pred, y_t)

opt.zero_grad()
loss.backward()

opt.step()

In [22]:
loss.item()

1043.4547119140625

In [23]:
for epoch in range(1, 10001):
    y_pred = regression(x_t)
    loss = criterion(y_pred, y_t)

    opt.zero_grad()
    loss.backward()

    opt.step()

    if epoch % 100 == 0:
        print(f"epoch: {[epoch]} loss:{loss.item()}")

epoch: [100] loss:203.4430694580078
epoch: [200] loss:69.84640502929688
epoch: [300] loss:59.75893783569336
epoch: [400] loss:57.23213577270508
epoch: [500] loss:55.362525939941406
epoch: [600] loss:53.775753021240234
epoch: [700] loss:52.38758850097656
epoch: [800] loss:51.153934478759766
epoch: [900] loss:50.039451599121094
epoch: [1000] loss:49.01557159423828
epoch: [1100] loss:48.060787200927734
epoch: [1200] loss:47.1601676940918
epoch: [1300] loss:46.30411911010742
epoch: [1400] loss:45.48698043823242
epoch: [1500] loss:44.705684661865234
epoch: [1600] loss:43.95865249633789
epoch: [1700] loss:43.2449836730957
epoch: [1800] loss:42.56388473510742
epoch: [1900] loss:41.9144172668457
epoch: [2000] loss:41.295310974121094
epoch: [2100] loss:40.704978942871094
epoch: [2200] loss:40.141544342041016
epoch: [2300] loss:39.602909088134766
epoch: [2400] loss:39.086830139160156
epoch: [2500] loss:38.590999603271484
epoch: [2600] loss:38.11316680908203
epoch: [2700] loss:37.651145935058594


In [24]:
x_test_tensor = t.from_numpy(x_test).float()


In [25]:
y_test[54:57], regression(x_test_tensor[54:57])

(array([34.9, 26.6,  7.2]),
 tensor([[33.1914],
         [26.9242],
         [19.4426]], grad_fn=<AddmmBackward>))